In [1]:
from multiprocessing import Pool, cpu_count
import gc; gc.enable()
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn import *
import sklearn
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression

In [25]:
# reading train csv
wsdm_data = pd.read_csv(r"D:\Work_Masterarbeit\wsdm\train_v2.csv")

# reading members csv
members_data = pd.read_csv(r"D:\Work_Masterarbeit\wsdm\members_v3.csv")

# merging train dataset with members dataset
wsdm_members = pd.merge(wsdm_data, members_data, on='msno', how='left')
# getting the head (top 5 rows) of df
wsdm_members.head()

,msno,is_churn,city,bd,gender,registered_via,registration_init_time
0,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,male,3.0,20131223.0
1,f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=,1,13.0,20.0,male,3.0,20131223.0
2,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,1,13.0,18.0,male,3.0,20131227.0
3,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,1,1.0,0.0,NaN,7.0,20140109.0
4,K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,1,13.0,35.0,female,7.0,20140125.0


In [26]:
### 4.1 Loading files to merge-

In [27]:
# train data
wsdm_data = pd.read_csv(r"D:\Work_Masterarbeit\wsdm\train_v2.csv")
# members data
members_data = pd.read_csv(r"D:\Work_Masterarbeit\wsdm\members_v3.csv")
# transactions data
transactions_data = pd.read_csv(r"D:\Work_Masterarbeit\wsdm\transactions_v2.csv")
# user logs data
logs_data = pd.read_csv(r"D:\Work_Masterarbeit\wsdm\user_logs_v2.csv")

In [28]:
### 4.2 Merging files together-
### 4.2.1 For train data-

In [29]:
# merging members data with train data
wsdm_members = pd.merge(wsdm_data, members_data, on='msno', how='left')
# merging transactions data with train members data
wsdm_mem_trans = pd.merge(wsdm_members, transactions_data, on='msno', how='left')
# merging user logs data with train members transaction data
wsdm_dataset = pd.merge(wsdm_mem_trans, logs_data, on='msno', how='left')
# getting head of train dataset
pd.set_option('display.max_columns', 100)
wsdm_dataset.head()

,msno,is_churn,city,bd,gender,registered_via,registration_init_time,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
0,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,male,3.0,20131223.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20170305.0,7.0,0.0,3.0,5.0,71.0,68.0,17599.893
1,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,male,3.0,20131223.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20170301.0,138.0,19.0,7.0,1.0,21.0,158.0,8830.433
2,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,male,3.0,20131223.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20170319.0,0.0,0.0,0.0,0.0,34.0,17.0,7883.313
3,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,male,3.0,20131223.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20170316.0,15.0,0.0,0.0,1.0,38.0,17.0,9029.227
4,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,5.0,28.0,male,3.0,20131223.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20170310.0,0.0,0.0,0.0,0.0,8.0,8.0,1870.110


In [30]:
wsdm_dataset.shape

(16887877, 23)

In [ ]:
## Sample 50% of Data due to memory restrictions
wsdm_dataset = wsdm_dataset.sample(frac=.5)

In [32]:
# since registration_init_time contains a lot of nan values so impute median date
wsdm_dataset['registration_init_time'] = wsdm_dataset['registration_init_time'].fillna(20150204.0)
# getting registration_init_time array
reg_date = wsdm_dataset['registration_init_time'].values
reg_date

array([20150913., 20111114., 20150701., ..., 20060329., 20110131.,
       20150629.])

In [33]:
len(reg_date)

11821514

In [34]:
# storing these train registrationn dates to npz file
np.savez_compressed(r"D:\Work_Masterarbeit\wsdm\preprocessed_2\wsdm_registration_date", reg_date)

<h3> Vorverarbeitung
   

In [35]:
def preprocess(data):
    '''This preprocess function is used to perform basic preprocessing on top of train and test dataset
    These preprocessing consist outliers removal, nan imputation and replacing the values'''
    
    # imputing 0 in place of nan values in the city column
    data['city'] = data['city'].fillna(0)

    # removing outliers
    data['bd'] = data['bd'].apply(lambda x: x if (x < 72.0) and (x > 0.0) else np.nan)
    # imputing 28 as age instead of nan
    data['bd'] = data['bd'].fillna(28.0)

    # replacing male with 1 in gender
    data['gender'] = data['gender'].replace(to_replace='male', value=1)
    # replacing male with 2 in gender
    data['gender'] = data['gender'].replace(to_replace='female', value=2)
    # replacing nan with 0 in gender
    data['gender'] = data['gender'].fillna(0)

    # replace 0 instead of nan in registered_via
    data['registered_via'] = data['registered_via'].fillna(0)

    # filling median date in place of nan in the df
    data['registration_init_time'] = data['registration_init_time'].fillna(20150204.0)
    # converting float date to datetime
    data['registration_init_time'] = pd.to_datetime(data['registration_init_time'], format='%Y%m%d')

    # imputing 0 in place of nan value in payment_method_id
    data['payment_method_id'] = data['payment_method_id'].fillna(0)

    # removing outliers
    data['payment_plan_days'] = data['payment_plan_days'].apply(lambda x: x if (x <= 30.0) else np.nan)
    # imputing 30 in place of nan in payment_plan_days
    data['payment_plan_days'] = data['payment_plan_days'].fillna(30.0)

    # removing outliers
    data['plan_list_price'] = data['plan_list_price'].apply(lambda x: x if (x <= 180.0) else np.nan)
    # imputing 149 in place of nan in plan_list_price
    data['plan_list_price'] = data['plan_list_price'].fillna(149.0)

    # removing outliers
    data['actual_amount_paid'] = data['actual_amount_paid'].apply(lambda x: x if (x <= 180.0) else np.nan)
    # imputing 149 in place of nan in actual_amount_paid
    data['actual_amount_paid'] = data['actual_amount_paid'].fillna(149.0)

    # imputing 2 in place of nan values in is_auto_renew
    data['is_auto_renew'] = data['is_auto_renew'].fillna(2)

    # filling median date in place of nan in the df
    data['transaction_date'] = data['transaction_date'].fillna(20170316.0)
    # converting float date to datetime
    data['transaction_date'] = pd.to_datetime(data['transaction_date'], format='%Y%m%d')

    # filling median date in place of nan in the df
    data['membership_expire_date'] = data['membership_expire_date'].fillna(20170419.0)
    # converting float date to datetime
    data['membership_expire_date'] = pd.to_datetime(data['membership_expire_date'], format='%Y%m%d')

    # imputing 2 in place of nan values in is_cancel
    data['is_cancel'] = data['is_cancel'].fillna(2)

    # filling median date in place of nan in the df
    data['date'] = data['date'].fillna(20170316.0)
    # converting float date to datetime
    data['date'] = pd.to_datetime(data['date'], format='%Y%m%d')

    # removing outliers
    data['num_25'] = data['num_25'].apply(lambda x: x if (x <= 15.0) else np.nan)
    # now I can impute 2 instead of nan in num_25
    data['num_25'] = data['num_25'].fillna(2.0)

    # removing outliers
    data['num_50'] = data['num_50'].apply(lambda x: x if (x <= 4.0) else np.nan)
    # now I can impute 0 instead of nan in num_50
    data['num_50'] = data['num_50'].fillna(0)

    # removing outliers
    data['num_75'] = data['num_75'].apply(lambda x: x if (x <= 3.0) else np.nan)
    # now I can impute 0 instead of nan in num_75
    data['num_75'] = data['num_75'].fillna(0)

    # removing outliers
    data['num_985'] = data['num_985'].apply(lambda x: x if (x <= 3.0) else np.nan)
    # now I can impute 0 instead of nan in num_985
    data['num_985'] = data['num_985'].fillna(0)

    # removing outliers
    data['num_100'] = data['num_100'].apply(lambda x: x if (x <= 74.0) else np.nan)
    # now I can impute 14 instead of nan in num_100
    data['num_100'] = data['num_100'].fillna(14.0)

    # removing outliers
    data['num_unq'] = data['num_unq'].apply(lambda x: x if (x <= 68.0) else np.nan)
    # now I can impute 16 instead of nan in num_unq
    data['num_unq'] = data['num_unq'].fillna(16.0)

    # removing outliers
    data['total_secs'] = data['total_secs'].apply(lambda x: x if (x <= 19167.549700000025) else np.nan)
    # now I can impute 3880.765 instead of nan in total_secs
    data['total_secs'] = data['total_secs'].fillna(3880.765)

In [36]:
# Checking null values for train data
# getting features wise null values (number)
wsdm_dataset.isnull().sum()

msno                            0
is_churn                        0
city                        81908
bd                          81908
gender                    5503216
registered_via              81908
registration_init_time          0
payment_method_id          238502
payment_plan_days          238502
plan_list_price            238502
actual_amount_paid         238502
is_auto_renew              238502
transaction_date           238502
membership_expire_date     238502
is_cancel                  238502
date                       174409
num_25                     174409
num_50                     174409
num_75                     174409
num_985                    174409
num_100                    174409
num_unq                    174409
total_secs                 174409
dtype: int64

In [37]:
# performing preprocessing on top of entire train data
preprocess(wsdm_dataset)

In [38]:
# again just to cross check if there is any null or not
wsdm_dataset.isnull().values.any()

False

In [39]:
# checking for any duplicate row
wsdm_dataset.duplicated().any()

False

In [40]:
# saving train file to disk
wsdm_dataset.to_csv(r"D:\Work_Masterarbeit\wsdm\preprocessed_2\train.csv", index=False)

<h2>Feature Engineering

In [41]:
# loading the files
wsdm_dataset = pd.read_csv(r"D:\Work_Masterarbeit\wsdm\preprocessed_2\train.csv")

In [42]:
%%time
# Feature - 4 to 12 (sum based features)
# Feature - 13 to 19 (mean based features)
# Feature - 20 to 27 (standard deviation based features)
# Feature - 28 (nunique based feature)
# Feature - 29 and 30 (min and max based features)
# Feature - 31 to 33 (mean based features for transaction)
# Feature - 34 (transaction count)
# Feature - 35 (transaction date max)
# Feature - 36 (membership expiry date max)
# Feature - 37 (membership expiry date count)

def std(x):
    '''finding standard deviation using numpy,
    to avoid getting nan values'''
    return np.std(x)

# grouping them together for train data
temp_df_train = wsdm_dataset.groupby('msno').agg(num_25_sum=('num_25', 'sum'),
                                num_50_sum=('num_50', 'sum'),
                                num_75_sum=('num_75', 'sum'),
                                num_985_sum=('num_985', 'sum'),
                                num_100_sum=('num_100', 'sum'),
                                num_unq_sum=('num_unq', 'sum'),
                                total_secs_sum=('total_secs', 'sum'),
                                num_25_mean=('num_25', 'mean'),
                                num_50_mean=('num_50', 'mean'),
                                num_75_mean=('num_75', 'mean'),
                                num_985_mean=('num_985', 'mean'),
                                num_100_mean=('num_100', 'mean'),
                                num_unq_mean=('num_unq', 'mean'),
                                total_secs_mean=('total_secs', 'mean'),
                                num_25_std=('num_25', std),
                                num_50_std=('num_50', std),
                                num_75_std=('num_75', std),
                                num_985_std=('num_985', std),
                                num_100_std=('num_100', std),
                                num_unq_std=('num_unq', std),
                                total_secs_std=('total_secs', std),
                                active_days=('date', 'nunique'),
                                date_min=('date', 'min'),
                                date_max=('date', 'max'),
                                payment_plan_days_mean=('payment_plan_days', 'mean'),
                                plan_list_price_mean=('plan_list_price', 'mean'),
                                actual_amount_paid_mean=('actual_amount_paid', 'mean'),
                                transaction_date_count=('transaction_date', 'nunique'),
                                transaction_date_max=('transaction_date', 'max'),
                                membership_expire_date_max=('membership_expire_date', 'max'),
                                membership_expire_count=('membership_expire_date', 'nunique'))
# merging them with the wsdm dataset
wsdm_dataset = pd.merge(wsdm_dataset, temp_df_train, on='msno', how='left')

Wall time: 6min 45s


In [43]:
# reference: https://medium.com/@vincentteyssier/optimizing-the-size-of-a-pandas-dataframe-for-low-memory-environment-5f07db3d72e
# reference: https://numpy.org/doc/stable/reference/generated/numpy.iinfo.html
# reference: https://numpy.org/doc/stable/reference/generated/numpy.finfo.html

def datatype_changer(dataset):
    # iterating through all the columns in the dataframe
    for col in dataset.columns:
        # getting column's datatype
        col_type = dataset[col].dtype
        
        # checking if datatype of column is 'object' or not
        # if column type is not object
        if (col_type == int) or (col_type == float):
            # getting minimum value of a column
            min_val = dataset[col].min()
            # getting maximum value of a column
            max_val = dataset[col].max()
            # checking whether the datatype contain first 3 characters as int or not, if int
            if str(col_type)[:3] == 'int':
                # cheking the minimal and maximal value for int8, int16, int32 and int64 in numpy
                if min_val > np.iinfo(np.int8).min and max_val < np.iinfo(np.int8).max:
                    dataset[col] = dataset[col].astype(np.int8)
                elif min_val > np.iinfo(np.int16).min and max_val < np.iinfo(np.int16).max:
                    dataset[col] = dataset[col].astype(np.int16)
                elif min_val > np.iinfo(np.int32).min and max_val < np.iinfo(np.int32).max:
                    dataset[col] = dataset[col].astype(np.int32)
                else:
                    dataset[col] = dataset[col].astype(np.int64)
            else:
                # if it is non int, which is ultimately float
                # cheking the minimal and maximal value for float16, float32 and float64 in numpy
                if min_val > np.finfo(np.float16).min and max_val < np.finfo(np.float16).max:
                    dataset[col] = dataset[col].astype(np.float16)
                elif min_val > np.finfo(np.float32).min and max_val < np.finfo(np.float32).max:
                    dataset[col] = dataset[col].astype(np.float32)
                else:
                    dataset[col] = dataset[col].astype(np.float64)
        else:
            # keeping rest of them to category datatype instead of object
            dataset[col] = dataset[col].astype('category')
            
    # returning head of the dataframe
    return dataset

In [44]:
wsdm_dataset = datatype_changer(wsdm_dataset)

In [45]:
# saving train file
wsdm_dataset.to_csv(r"D:\Work_Masterarbeit\wsdm\preprocessed_2\wsdm_FA.csv", index=False)

<h1>3. Prepare Data for Modelling

In [17]:
# loading train dataset
wsdm_dataset = pd.read_csv(r"D:\Work_Masterarbeit\wsdm\preprocessed_2\wsdm_FA.csv")

In [47]:
# removing Rawdata Features
# wsdm_dataset = wsdm_dataset.drop(['registration_init_time',
#                               'transaction_date', 
#                               'membership_expire_date', 
#                               'date', 
#                               'num_25',
#                               'num_50',
#                               'num_75',
#                               'num_985',
#                               'num_100',
#                               'num_unq',
#                               'total_secs',
#                               'is_weekend',
#                               'is_weekday',
#                               'day_of_the_week', 
#                               'date_min', 
#                               'date_max', 
#                               'transaction_date_max', 
#                               'membership_expire_date_max'], axis=1, inplace=False)

In [48]:
# adding registration init time feature to train dataset
# load the registration array
reg_date = np.load(r"D:\Work_Masterarbeit\wsdm\preprocessed_2\wsdm_registration_date.npz")
# display the array
reg_date['arr_0']

array([20150913., 20111114., 20150701., ..., 20060329., 20110131.,
       20150629.])

In [49]:
# converting numpy array to list
registration_init_time = reg_date['arr_0'].tolist()
# getting length of the list
len(registration_init_time)

11821514

In [50]:
# creating new feature, and adding it to train dataset
wsdm_dataset['registration_init_time'] = registration_init_time

In [51]:
# saving new dataset
wsdm_dataset.to_csv(r"D:\Work_Masterarbeit\wsdm\preprocessed_2\prepared_train_incraw.csv", index=False)

In [52]:
# reading csv file for removing duplicates
wsdm_dataset = pd.read_csv(r"D:\Work_Masterarbeit\wsdm\preprocessed_2\prepared_train_incraw.csv")
# getting all columns of train dataset
wsdm_dataset.columns

Index(['msno', 'is_churn', 'city', 'bd', 'gender', 'registered_via',
       'registration_init_time', 'payment_method_id', 'payment_plan_days',
       'plan_list_price', 'actual_amount_paid', 'is_auto_renew',
       'transaction_date', 'membership_expire_date', 'is_cancel', 'date',
       'num_25', 'num_50', 'num_75', 'num_985', 'num_100', 'num_unq',
       'total_secs', 'num_25_sum', 'num_50_sum', 'num_75_sum', 'num_985_sum',
       'num_100_sum', 'num_unq_sum', 'total_secs_sum', 'num_25_mean',
       'num_50_mean', 'num_75_mean', 'num_985_mean', 'num_100_mean',
       'num_unq_mean', 'total_secs_mean', 'num_25_std', 'num_50_std',
       'num_75_std', 'num_985_std', 'num_100_std', 'num_unq_std',
       'total_secs_std', 'active_days', 'date_min', 'date_max',
       'payment_plan_days_mean', 'plan_list_price_mean',
       'actual_amount_paid_mean', 'transaction_date_count',
       'transaction_date_max', 'membership_expire_date_max',
       'membership_expire_count'],
      dtype='obje

In [53]:
# getting shape of train dataset
wsdm_dataset.shape

(11821514, 54)

In [54]:
# getting columns and converting them to list
cols = wsdm_dataset.columns.tolist()
# saving them for further use
np.savez_compressed(r"D:\Work_Masterarbeit\wsdm\preprocessed_2\columns_incraw.npz", cols)

In [55]:
# saving actual train dataset
wsdm_dataset.to_csv(r"D:\Work_Masterarbeit\wsdm\preprocessed_2\actual_train_incraw.csv", index=False)

<h2> 3.2 Train Test Split

In [3]:
# loading actual train data
wsdm_data = pd.read_csv(r"D:\Work_Masterarbeit\wsdm\preprocessed_2\actual_train_incraw.csv")

In [4]:
# replacing all nan and infinity to 0
wsdm_data = wsdm_data.replace(np.nan, 0, inplace=False)
wsdm_data = wsdm_data.replace(np.inf, 0, inplace=False)

In [5]:
## Sample 50% of Data due to memory restrictions
wsdm_data = wsdm_data.sample(frac=.5)

In [6]:
## Typkonvertierungen
wsdm_data['transaction_date_max'] = wsdm_data['transaction_date_max'].str.replace('-', '').astype(float)
wsdm_data['membership_expire_date'] = wsdm_data['membership_expire_date'].str.replace('-', '').astype(float)
wsdm_data['membership_expire_date_max'] = wsdm_data['membership_expire_date_max'].str.replace('-', '').astype(float)

wsdm_data['transaction_date'] = wsdm_data['transaction_date'].str.replace('-', '').astype(float)
wsdm_data['date'] = wsdm_data['date'].str.replace('-', '').astype(float)
wsdm_data['date_min'] = wsdm_data['date_min'].str.replace('-', '').astype(float)
wsdm_data['date_max'] = wsdm_data['date_max'].str.replace('-', '').astype(float)

In [7]:
# https://stackoverflow.com/a/29651514
def normalize(df):
    result1 = df.copy()
    for feature_name in df.columns:
        if (str(feature_name) != str('msno') and str(feature_name)!=str('is_churn')):
            max_value = df[feature_name].max()
            min_value = df[feature_name].min()
            result1[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result1

wsdm_dataset = normalize(wsdm_data)

In [8]:
# getting labels for y
labels = wsdm_dataset['is_churn'].values
labels

array([1, 1, 0, ..., 0, 0, 0], dtype=int64)

In [9]:
# getting labels for X
data = wsdm_dataset.drop(['msno', 'is_churn'], axis=1, inplace=False)
data

,city,bd,gender,registered_via,registration_init_time,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,...,active_days,date_min,date_max,payment_plan_days_mean,plan_list_price_mean,actual_amount_paid_mean,transaction_date_count,transaction_date_max,membership_expire_date_max,membership_expire_count
5084845,0.045455,0.385714,0.0,0.538462,0.690658,1.000000,1.0,0.827778,0.827778,0.5,...,0.966667,0.000000,1.000000,1.000000,0.827778,0.827778,0.041667,0.999160,0.006338,0.009662
10230250,0.045455,0.214286,1.0,0.230769,0.773724,0.414634,1.0,0.000000,0.000000,0.0,...,0.733333,0.266667,1.000000,1.000000,0.490000,0.490000,0.041667,0.999110,0.006338,0.009662
8349976,0.045455,0.385714,0.0,0.538462,0.929818,1.000000,1.0,0.550000,0.550000,0.5,...,0.600000,0.066667,0.966667,1.000000,0.550000,0.550000,0.000000,0.998764,0.002938,0.000000
3226464,0.681818,0.328571,0.5,0.692308,0.156786,0.951220,1.0,0.827778,0.827778,0.5,...,0.766667,0.000000,1.000000,1.000000,0.827778,0.827778,0.020833,1.000000,0.004902,0.004831
357228,0.227273,0.257143,0.5,0.230769,0.846052,0.878049,1.0,1.000000,1.000000,0.0,...,0.700000,0.033333,0.966667,1.000000,1.000000,1.000000,0.000000,0.998715,0.002905,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4051007,0.045455,0.385714,0.0,0.538462,0.695196,1.000000,1.0,0.827778,0.661111,0.5,...,0.600000,0.000000,1.000000,0.893667,0.682222,0.732222,0.500000,0.999802,0.331529,0.125604
9292551,0.500000,0.428571,0.5,0.692308,0.229891,0.951220,1.0,0.827778,0.827778,0.5,...,0.766667,0.000000,1.000000,1.000000,0.827778,0.827778,0.020833,1.000000,0.004869,0.004831
9193532,0.590909,0.371429,1.0,0.692308,0.310573,0.878049,1.0,1.000000,1.000000,0.5,...,0.633333,0.000000,0.966667,1.000000,1.000000,1.000000,0.000000,0.999258,0.003086,0.000000
10151992,0.045455,0.385714,0.0,0.538462,0.696704,1.000000,1.0,0.550000,0.550000,0.5,...,0.000000,0.500000,0.500000,1.000000,0.550000,0.550000,0.000000,0.998616,0.002888,0.000000


In [10]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.3, random_state=42)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(4137529, 52) (4137529,)
(1773228, 52) (1773228,)


In [11]:
# converting X_train to numpy array
wsdm_arr = X_train.values
wsdm_arr

array([[0.04545455, 0.38571429, 0.        , ..., 0.99945623, 0.00316889,
        0.        ],
       [0.04545455, 0.38571429, 0.        , ..., 0.99851698, 0.00293783,
        0.        ],
       [0.59090909, 0.22857143, 1.        , ..., 0.99851698, 0.0028553 ,
        0.        ],
       ...,
       [0.04545455, 0.38571429, 0.        , ..., 0.99906075, 0.00302035,
        0.        ],
       [0.36363636, 0.3       , 1.        , ..., 1.        , 0.00457179,
        0.00483092],
       [0.04545455, 0.38571429, 0.        , ..., 1.        , 0.00333394,
        0.        ]])

In [12]:
# checking shape of train array
wsdm_arr.shape

(4137529, 52)

In [13]:
# saving it for further use
np.savez_compressed(r"D:\Work_Masterarbeit\wsdm\preprocessed_2\actual_train_incraw.npz", wsdm_arr)

In [14]:
# displaying y_train
wsdm_labels_arr = y_train
wsdm_labels_arr

array([0, 0, 1, ..., 0, 0, 0], dtype=int64)

In [15]:
# checking shape of train labels
wsdm_labels_arr.shape

(4137529,)

In [16]:
# saving it for further use
np.savez_compressed(r"D:\Work_Masterarbeit\wsdm\preprocessed_2\actual_wsdm_labels_incraw.npz", wsdm_labels_arr)

In [17]:
# converting X_test to numpy array
cv_arr = X_test.values
cv_arr

array([[1.        , 0.37142857, 1.        , ..., 0.99861585, 0.00290482,
        0.        ],
       [0.04545455, 0.38571429, 0.        , ..., 0.99965396, 0.00323491,
        0.        ],
       [0.04545455, 0.38571429, 0.        , ..., 1.        , 0.00333394,
        0.        ],
       ...,
       [0.04545455, 0.4       , 0.5       , ..., 0.99925849, 0.00315239,
        0.        ],
       [0.27272727, 0.37142857, 1.        , ..., 0.99881358, 0.00293783,
        0.        ],
       [0.18181818, 0.54285714, 0.5       , ..., 1.        , 0.17453663,
        0.09178744]])

In [18]:
# checking shape of cv array
cv_arr.shape

(1773228, 52)

In [19]:
# saving it for further use
np.savez_compressed(r"D:\Work_Masterarbeit\wsdm\preprocessed_2\actual_cv_incraw.npz", cv_arr)

In [20]:
# displaying y_test
cv_labels_arr = y_test
cv_labels_arr

array([0, 0, 0, ..., 1, 0, 0], dtype=int64)

In [21]:
# checking shape of cv labels array
cv_labels_arr.shape

(1773228,)

In [22]:
# saving it for further use
np.savez_compressed(r"D:\Work_Masterarbeit\wsdm\preprocessed_2\actual_cv_labels_incraw.npz", cv_labels_arr)

<h1> 5. Modelling

In [2]:
# loading library
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import log_loss

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb


In [3]:
# reference: https://machinelearningmastery.com/how-to-save-a-numpy-array-to-file-for-machine-learning/
X_train = np.load(r"D:\Work_Masterarbeit\wsdm\preprocessed_2\actual_train_incraw.npz")
y_train = np.load(r"D:\Work_Masterarbeit\wsdm\preprocessed_2\actual_wsdm_labels_incraw.npz")
X_test = np.load(r"D:\Work_Masterarbeit\wsdm\preprocessed_2\actual_cv_incraw.npz")
y_test = np.load(r"D:\Work_Masterarbeit\wsdm\preprocessed_2\actual_cv_labels_incraw.npz")


In [4]:
X_train = X_train['arr_0']
X_test = X_test['arr_0']
y_train = y_train['arr_0']
y_test = y_test['arr_0']

# getting shapes
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(4137529, 52) (4137529,)
(1773228, 52) (1773228,)


In [5]:
# loading columns
cols = np.load(r"D:\Work_Masterarbeit\wsdm\preprocessed_2\columns_incraw.npz")
# converting numpy array to python list
cols = cols['arr_0'].tolist()
# removing first two cols 'msno' and 'is_churn'
cols = cols[2:]

<h3> Logistic Regression

In [6]:
%%time

# fitting logistic regression model with best parameter values
lr_model = LogisticRegression(penalty='l2', tol=0.0001, C=10, solver='newton-cg', class_weight='balanced', n_jobs=-1)
lr_model.fit(X_train, y_train)

Wall time: 15min 10s


LogisticRegression(C=10, class_weight='balanced', n_jobs=-1, solver='newton-cg')

In [7]:
# Reference: https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/#:~:text=Saving%20Your%20Model-,Save%20Your%20Model%20with%20pickle,it%20to%20make%20new%20predictions.
# saving the trained logistic regression ML model
filename = r"D:\Work_Masterarbeit\wsdm\preprocessed_2\finalized_model_lr_incraw.sav"
joblib.dump(lr_model, filename)

['D:\\Work_Masterarbeit\\wsdm\\preprocessed_2\\finalized_model_lr_incraw.sav']

In [8]:
# loading the best model from disk
filename = r"D:\Work_Masterarbeit\wsdm\preprocessed_2\finalized_model_lr_incraw.sav"
loaded_model = joblib.load(filename)

<h3> Model Evaluation

In [9]:
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score

lr_predict=lr_model.predict(X_test)
lr_predict


print('Accuracy:', accuracy_score(y_test, lr_predict))
print('F1 score weighted:', f1_score(y_test, lr_predict,average='weighted'))
print('F1 score macro:', f1_score(y_test, lr_predict, average='macro'))
print('Recall:', recall_score(y_test, lr_predict,average='weighted'))
print('Precision:', precision_score(y_test, lr_predict,average='weighted'))
print('\n clasification report:\n', classification_report(y_test, lr_predict))
print('\n confussion matrix:\n',confusion_matrix(y_test, lr_predict))

Accuracy: 0.8361953454378117
F1 score weighted: 0.8597320170509911
F1 score macro: 0.6927865268652117
Recall: 0.8361953454378117
Precision: 0.9046498825165213

 clasification report:
               precision    recall  f1-score   support

           0       0.97      0.85      0.90   1591796
           1       0.36      0.75      0.48    181432

    accuracy                           0.84   1773228
   macro avg       0.66      0.80      0.69   1773228
weighted avg       0.90      0.84      0.86   1773228


 confussion matrix:
 [[1347145  244651]
 [  45812  135620]]


In [10]:
## Logloss
# predicting probabilities for X_cv
pred_test = loaded_model.predict_proba(X_test)
# getting probabilities corresponding to class label 1 only
pred_test = pred_test[:,1]
# calculating log loss
print('Test log-loss: ', log_loss(y_test, pred_test))

Test log-loss:  0.4634192368521652


<h2> Decision Tree

In [12]:
%%time

# training a decision tree classifier model with best parameters
dtc_model = DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=5, min_samples_split=3, min_samples_leaf=1)
dtc_model.fit(X_train, y_train)

Wall time: 1min 10s


DecisionTreeClassifier(max_depth=5, min_samples_split=3)

In [13]:
# https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/#:~:text=Saving%20Your%20Model-,Save%20Your%20Model%20with%20pickle,it%20to%20make%20new%20predictions.
# saving the decision tree classifier model
filename = r"D:\Work_Masterarbeit\wsdm\preprocessed_2\finalized_model_dtc_incraw.sav"
joblib.dump(dtc_model, filename)

['D:\\Work_Masterarbeit\\wsdm\\preprocessed_2\\finalized_model_dtc_incraw.sav']

In [14]:
# loading the best decision tree classifier model from disk
filename = r"D:\Work_Masterarbeit\wsdm\preprocessed_2\finalized_model_dtc_incraw.sav"
loaded_model = joblib.load(filename)

<h3> Model Evaluation

In [15]:
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score

dtc_modelpred=dtc_model.predict(X_test)
dtc_modelpred


print('Accuracy:', accuracy_score(y_test, dtc_modelpred))
print('F1 score weighted:', f1_score(y_test, dtc_modelpred,average='weighted'))
print('F1 score macro:', f1_score(y_test, dtc_modelpred, average='macro'))
print('Recall:', recall_score(y_test, dtc_modelpred,average='weighted'))
print('Precision:', precision_score(y_test, dtc_modelpred,average='weighted'))
print('\n clasification report:\n', classification_report(y_test, dtc_modelpred))
print('\n confussion matrix:\n',confusion_matrix(y_test, dtc_modelpred))

Accuracy: 0.9626404500718464
F1 score weighted: 0.9614834330563252
F1 score macro: 0.8920651684509531
Recall: 0.9626404500718464
Precision: 0.9612149001144429

 clasification report:
               precision    recall  f1-score   support

           0       0.97      0.99      0.98   1591796
           1       0.86      0.75      0.80    181432

    accuracy                           0.96   1773228
   macro avg       0.92      0.87      0.89   1773228
weighted avg       0.96      0.96      0.96   1773228


 confussion matrix:
 [[1570426   21370]
 [  44877  136555]]


In [16]:
## Logloss
# predicting probabilities for X_cv
pred_test = loaded_model.predict_proba(X_test)
# getting probabilities corresponding to class label 1 only
pred_test = pred_test[:,1]
# calculating log loss
print('Test log-loss: ', log_loss(y_test, pred_test))

Test log-loss:  0.1253341464733862


<h2> Random Forest

In [17]:
%%time

# training the random forest classifier for best hyper parameters
rf_model = RandomForestClassifier(n_estimators=500, criterion='gini', max_depth=15, min_samples_split=3, min_samples_leaf=2, max_features='sqrt', bootstrap=False, n_jobs=-1)
rf_model.fit(X_train, y_train)

Wall time: 36min 38s


RandomForestClassifier(bootstrap=False, max_depth=15, max_features='sqrt',
                       min_samples_leaf=2, min_samples_split=3,
                       n_estimators=500, n_jobs=-1)

In [18]:
# https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/#:~:text=Saving%20Your%20Model-,Save%20Your%20Model%20with%20pickle,it%20to%20make%20new%20predictions.
# saving the decision tree classifier model
filename = r"D:\Work_Masterarbeit\wsdm\preprocessed_2\finalized_model_rf_incraw.sav"
joblib.dump(rf_model, filename)

['D:\\Work_Masterarbeit\\wsdm\\preprocessed_2\\finalized_model_rf_incraw.sav']

In [19]:
# loading the best model from disk
filename = r"D:\Work_Masterarbeit\wsdm\preprocessed_2\finalized_model_rf_incraw.sav"
loaded_model = joblib.load(filename)

<h3>Model Evaluation

In [20]:
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score

rf_modelpred=rf_model.predict(X_test)
rf_modelpred


print('Accuracy:', accuracy_score(y_test, rf_modelpred))
print('F1 score weighted:', f1_score(y_test, rf_modelpred,average='weighted'))
print('F1 score macro:', f1_score(y_test, rf_modelpred, average='macro'))
print('Recall:', recall_score(y_test, rf_modelpred,average='weighted'))
print('Precision:', precision_score(y_test, rf_modelpred,average='weighted'))
print('\n clasification report:\n', classification_report(y_test, rf_modelpred))
print('\n confussion matrix:\n',confusion_matrix(y_test, rf_modelpred))

Accuracy: 0.9789282596485054
F1 score weighted: 0.978328114519355
F1 score macro: 0.9394031623497037
Recall: 0.9789282596485054
Precision: 0.9785531199361152

 clasification report:
               precision    recall  f1-score   support

           0       0.98      1.00      0.99   1591796
           1       0.95      0.84      0.89    181432

    accuracy                           0.98   1773228
   macro avg       0.97      0.92      0.94   1773228
weighted avg       0.98      0.98      0.98   1773228


 confussion matrix:
 [[1583986    7810]
 [  29555  151877]]


In [21]:
## Logloss
# predicting probabilities for X_cv
pred_test = loaded_model.predict_proba(X_test)
# getting probabilities corresponding to class label 1 only
pred_test = pred_test[:,1]
# calculating log loss
print('Test log-loss: ', log_loss(y_test, pred_test))

Test log-loss:  0.0701262256468004
